# How to Grid Search Hyperparameters for Deep Learning Models in Python With Keras

    * I apply this guidelines for my project, working with wine quality prediction database

Grid search is a model hyperparameter optimization technique. In scikit-learn this technique is provided in the GridSearchCV class. By default, accuracy is the score that is optimized, but other scores can be specified in the score argument of the GridSearchCV constructor.



In [1]:
# load libraries
import matplotlib.pyplot as plt    
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, KFold

from tensorflow import keras 
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.regularizers import l2

from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import SGD

from scikeras.wrappers import KerasClassifier
from scikeras.wrappers import KerasRegressor

In [25]:
#Import and standardize database

df = pd.read_csv("wine.csv", index_col = "index")
X = df.iloc[:, 0:11]
df['quality'] = df['quality'].apply(lambda x: 0 if x < 6 else 1)
y = df.iloc[:, 11]
print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.30, stratify=y)
# creating normalization object 
# fit data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

(6497, 11)
(6497,)


## Use Grid Search in scikit-learn

### 1. Tune batch size and training epochs.

In [27]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(22, input_dim=11, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
# fix random seed for reproducibility
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters: BATCH_SIZE and EPOCHS
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs) #define parameters here

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/2679263144.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.772156 using {'batch_size': 80, 'epochs': 50}
0.765777 (0.016480) with: {'batch_size': 10, 'epochs': 10}
0.771937 (0.012120) with: {'batch_size': 10, 'epochs': 50}
0.759842 (0.009972) with: {'batch_size': 10, 'epochs': 100}
0.761818 (0.015909) with: {'batch_size': 20, 'epochs': 10}
0.767758 (0.015183) with: {'batch_size': 20, 'epochs': 50}
0.767101 (0.013513) with: {'batch_size': 20, 'epochs': 100}
0.758960 (0.017965) with: {'batch_size': 40, 'epochs': 10}
0.771057 (0.009727) with: {'batch_size': 40, 'epochs': 50}
0.760943 (0.012870) with: {'batch_size': 40, 'epochs': 100}
0.757421 (0.014494) with: {'batch_size': 60, 'epochs': 10}
0.766217 (0.013255) with: {'batch_size': 60, 'epochs': 50}
0.765782 (0.014891) with: {'batch_size': 60, 'epochs': 100}
0.752582 (0.009767) with: {'batch_size': 80, 'epochs': 10}
0.772156 (0.012269) with: {'batch_size': 80, 'epochs': 50}
0.767760 (0.007015) with: {'batch_size': 80, 'epochs': 100}
0.754564 (0.014816) with: {'batch_size': 100, 'epochs': 

### 2. Tune optimization algorithms

In [28]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=11, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility

model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=80, verbose=0)

# define the grid search Parameters ###OPTIMIZERS
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)   ##Define changes here
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/3687319543.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=80, verbose=0)
2021-09-09 11:13:59.650488: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 11:13:59.651394: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 11:13:5

Best: 0.763798 using {'optimizer': 'RMSprop'}
0.744665 (0.016036) with: {'optimizer': 'SGD'}
0.763798 (0.014067) with: {'optimizer': 'RMSprop'}
0.643495 (0.025556) with: {'optimizer': 'Adagrad'}
0.517694 (0.091624) with: {'optimizer': 'Adadelta'}
0.761159 (0.017830) with: {'optimizer': 'Adam'}
0.758521 (0.017875) with: {'optimizer': 'Adamax'}
0.759621 (0.018063) with: {'optimizer': 'Nadam'}


### 3. Tune learning rate and momentum

In [29]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(22, input_dim=11, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    optimizer = SGD(lr=learning_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
# define the grid search parameters
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learning_rate=learning_rate, momentum=momentum)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/1875075457.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
2021-09-09 11:33:11.282743: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 11:33:11.282749: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 11:33:

Best: 0.768858 using {'learning_rate': 0.001, 'momentum': 0.9}
0.752803 (0.020415) with: {'learning_rate': 0.001, 'momentum': 0.0}
0.747743 (0.015601) with: {'learning_rate': 0.001, 'momentum': 0.2}
0.755003 (0.014602) with: {'learning_rate': 0.001, 'momentum': 0.4}
0.755663 (0.005719) with: {'learning_rate': 0.001, 'momentum': 0.6}
0.762919 (0.015904) with: {'learning_rate': 0.001, 'momentum': 0.8}
0.768858 (0.007691) with: {'learning_rate': 0.001, 'momentum': 0.9}
0.765120 (0.017264) with: {'learning_rate': 0.01, 'momentum': 0.0}
0.766219 (0.012790) with: {'learning_rate': 0.01, 'momentum': 0.2}
0.764020 (0.013197) with: {'learning_rate': 0.01, 'momentum': 0.4}
0.765998 (0.010722) with: {'learning_rate': 0.01, 'momentum': 0.6}
0.757642 (0.005719) with: {'learning_rate': 0.01, 'momentum': 0.8}
0.746208 (0.007497) with: {'learning_rate': 0.01, 'momentum': 0.9}
0.765341 (0.011336) with: {'learning_rate': 0.1, 'momentum': 0.0}
0.756323 (0.019120) with: {'learning_rate': 0.1, 'momentum': 

### 4. Tune network weight initialization

In [8]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=11, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode= init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/1334812496.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)


Best: 0.772945 using {'init_mode': 'lecun_uniform'}
0.763902 (0.004660) with: {'init_mode': 'uniform'}
0.772945 (0.005743) with: {'init_mode': 'lecun_uniform'}
0.765634 (0.003298) with: {'init_mode': 'normal'}
0.628052 (0.008115) with: {'init_mode': 'zero'}
0.765442 (0.003284) with: {'init_mode': 'glorot_normal'}
0.768327 (0.004062) with: {'init_mode': 'glorot_uniform'}
0.772560 (0.005341) with: {'init_mode': 'he_normal'}
0.771214 (0.005833) with: {'init_mode': 'he_uniform'}


### 5. Tune activation functions

In [31]:
# Use scikit-learn to grid search the dropout rate
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=11, kernel_initializer='uniform', activation=activation))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/4246544714.py:25: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
2021-09-09 12:25:45.968742: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 12:25:45.968780: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 12:25:

Best: 0.771497 using {'activation': 'tanh'}
0.766880 (0.013339) with: {'activation': 'softmax'}
0.760277 (0.021530) with: {'activation': 'softplus'}
0.767538 (0.013461) with: {'activation': 'softsign'}
0.756322 (0.013529) with: {'activation': 'relu'}
0.771497 (0.008779) with: {'activation': 'tanh'}
0.763580 (0.014065) with: {'activation': 'sigmoid'}
0.757203 (0.008618) with: {'activation': 'hard_sigmoid'}
0.759400 (0.017341) with: {'activation': 'linear'}


### 6. Tune dropout regularization

In [30]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=11, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=40, verbose=0)

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/809466583.py:24: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=40, verbose=0)


Best: 0.765338 using {'dropout_rate': 0.1, 'weight_constraint': 5}
0.757641 (0.019995) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.764679 (0.019234) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.762699 (0.018961) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.764019 (0.017586) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.762699 (0.015499) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.754782 (0.020414) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.764238 (0.017329) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.761820 (0.017669) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.764679 (0.014294) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.765338 (0.020782) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.756981 (0.020938) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.763360 (0.017234) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.760501 (0.015114) with: {'dropout_rate': 0.2, 'weight_constraint': 

/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/te

/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/Users/minhhienvo/opt/miniconda3/envs/te

### 7. Tune the number of neurons in the hidden layer.

In [22]:
# Use scikit-learn to grid search the number of neurons
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import Adam
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=11, kernel_initializer='uniform', activation='relu', kernel_constraint=maxnorm(4)))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    #model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=40, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 22, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/var/folders/nx/48r1p7j10ys5bbndt2lmfpk40000gp/T/ipykernel_13765/1691992900.py:27: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=40, verbose=0)


Best: 0.755819 using {'neurons': 30}
0.740233 (0.007913) with: {'neurons': 1}
0.748699 (0.008631) with: {'neurons': 5}
0.753510 (0.005961) with: {'neurons': 10}
0.752739 (0.011311) with: {'neurons': 15}
0.751779 (0.005033) with: {'neurons': 20}
0.753896 (0.003984) with: {'neurons': 22}
0.751585 (0.009374) with: {'neurons': 25}
0.755819 (0.005052) with: {'neurons': 30}
